# Gridded EO Data

> Standardizing to H3 global grid indexing when working with rasters. For this example series, we focus on B04 (red), B03 (green), B02 (blue), and B08 (nir).

---
__Last Update:__ 16 JAN 2024 [Mosaic 0.3.14]

## Setup Notes

> Raster processing is memory intensize. Here are some config tips.

* This notebook was run on AWS [r5d.4xlarge](https://www.databricks.com/product/pricing/product-pricing/instance-types) instances (1-10 workers auto-scaling for up to 160 concurrent tasks).
* __Optional:__ Prior to launching, the following can be added to the cluster spark configs, adapted from this databricks [blog](https://www.databricks.com/blog/2015/05/28/tuning-java-garbage-concept_set-for-spark-applications.html) and these JVM tuning [docs](https://docs.oracle.com/cd/E40972_01/doc.70/e40973/cnf_jvmgc.htm#autoId2): `spark.executor.extraJavaOptions -XX:+UseG1GC -XX:+PrintFlagsFinal -XX:InitiatingHeapOccupancyPercent=35 -XX:G1HeapRegionSize=32M -XX:ParallelGCThreads=16 -XX:ConcGCThreads=4`. _This optimization is not required for this example._

## Imports + Config

In [0]:
%pip install --quiet 'databricks-mosaic<0.4,>=0.3'
%pip install --quiet rasterio==1.3.5 gdal==3.4.3 pystac pystac_client planetary_computer tenacity rich pandas==1.5.3

In [0]:
# -- configure AQE for more compute heavy operations
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False) # <- option-2: just tweak partition management
spark.conf.set("spark.sql.shuffle.partitions", 512)

# -- import databricks + delta + spark functions
from delta.tables import *
from pyspark.databricks.sql import functions as dbf
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

# -- setup mosaic
import mosaic as mos

mos.enable_mosaic(spark, dbutils)
mos.enable_gdal(spark)

# -- other imports
from datetime import datetime
from io import BytesIO
from matplotlib import pyplot
from rasterio.io import MemoryFile

import library
import os
import pathlib
import planetary_computer
import pystac_client
import rasterio
import requests
import warnings

warnings.simplefilter("ignore")

In [0]:
mos.__version__

In [0]:
%reload_ext autoreload
%autoreload 2
%reload_ext library

### Databricks Catalog + Schema

> This is for reading and writing out table(s). __Note: these should already exists after running notebook 01.__

In [0]:
# adjust to match your catalog + schema
catalog_name = "geospatial_docs"
db_name = "eo_alaska"

sql(f"""USE CATALOG {catalog_name}""")
sql(f"""USE DATABASE {db_name}""")

## Data load

We can easily browse the data we have downloaded previously (notebook 01). The download metadata is stored as a delta table.

In [0]:
%sql show tables

__Prepare initial band table for eventual tessellation.__

In [0]:
def gen_tiled_band_tbl(
  band_df, tbl_name, tile_threshold=32, repartition_factor = 3,
  item_id_col="item_id", file_col="out_file_path", size_col="size", bbox_col="bbox",
  drop_cols=["timestamp", "h3_set", "asset", "item_properties", "item_concept_set", "item_bbox", "stac_version", 
        "last_update", "out_dir_fuse", "out_filename", "out_file_path", "out_file_sz", "is_out_file_valid"],
  do_try_open=False, do_display=True, do_overwrite=False, do_append=False
):
  """
  Generate a tile table for a band dataframe.
  - tile_threshold: size in MB (default 32MB)
  - if you have some bad data, set do_try_open=True
  Returns the resulting dataframe (from the spark table generated).
  """
  spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
  spark.conf.set("spark.sql.shuffle.partitions", 512)
  
  if do_overwrite:
    sql(f"""drop table if exists {tbl_name}""")
  if not spark.catalog.tableExists(tbl_name) or do_append:
    _band_df = (
      band_df
        .repartition(round(band_df.count() / repartition_factor))
        .withColumn(
          "tile",
          mos.rst_fromfile(file_col, F.lit(tile_threshold))
        )   
    )
    if do_try_open:
      _band_df = _band_df.filter(mos.rst_tryopen("tile"))
    (
      _band_df
        .withColumn(size_col, mos.rst_memsize("tile"))
        .withColumn("tile", mos.rst_initnodata("tile"))
        .withColumn(bbox_col, mos.rst_boundingbox("tile"))
        .withColumn("srid", col("item_properties").getItem("proj:epsg"))
      .drop(*drop_cols)
      .write
        .mode("append")
        .saveAsTable(tbl_name)
    )
  sql(f"""OPTIMIZE {tbl_name} ZORDER BY ({item_id_col})""")
  tile_df = spark.table(tbl_name)
  if do_display:
    print(f"\n-- {tbl_name}")
    print(f"count? {tile_df.count():,}")
    tile_df.limit(10).display()
  return tile_df

In [0]:
# set date as variable for consistency across bands.
# - you don't have to filter
filter_date = "2021-06-01"

__Filter `band_b02` (blue - 10m) to a single date (2021-06-01) from all available dates.__

In [0]:
b02_df = (
  spark.read.table("band_b02")
  .filter(col('date') == F.lit(filter_date))
)
print(f"count? {b02_df.count():,}")
b02_df.limit(1).show(vertical=True) # <- show + limiting for ipynb only

For the purpose of raster data analysis mosaic framework provides distributed gdal data readers. While we can start by loading the underlying files directly, since we have the band tables, we use `rst_fromfile` to load 'out_file_path' and new columns from the following:

* `rst_fromfile` also subdivides the images (32 MB threshold) to make sure the imagery is balanced and more parallelised
* `rst_memsize` for 'size' 
* `rst_initnodata` to standardize the tile no data (avoids issues later in calcs)
* `rst_boundingbox` to generate tile bounding boxes
* 'srid' from the raster metadata

__Notes:__

1. Since this involves a bit of computation, we persist out as Delta Table to then use multiple times below.
1. Operation takes ~6 minutes using the configuration described at top of notebook.
1. We also OPTIMIZE (table layout) based on item_id which will help future ops.

In [0]:
tile_df = gen_tiled_band_tbl(b02_df, "band_b02_tile")

In [0]:
display (
  tile_df
    .select(mos.st_astext("bbox").alias("bbox_wkt"), "srid")
    .groupBy("bbox_wkt", "srid")
      .count()
  .limit(10)
)

## Initial Tile Plots

> These are samples from the full data.

_[1] Tile raster plot._

In [0]:
to_plot = tile_df.limit(10).collect()

In [0]:
library.plot_raster(to_plot[3]["tile"]["raster"])

_[2] Bounding box plot._

> Just for a subset of 'item_ids'. __Note: we specify the SRID for rendering in Mosaic__ since it is not 4326. We got this information earlier.

In [0]:
to_plot = (
  tile_df
    .filter(col("item_id").contains(F.lit("_R014_")))
    .select(mos.st_astext("bbox").alias("bbox_wkt"), "srid")
    .groupBy("bbox_wkt", "srid")
      .count()
    .orderBy(F.desc("count"))
)
print(f"count? {to_plot.count():,}")
to_plot.limit(5).display()

In [0]:
to_plot.groupBy("srid").count().orderBy("srid").display()

In [0]:
to_plot_32607 = to_plot.filter("srid = 32607")

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# to_plot_32607 bbox_wkt geometry(32607) 1_000

## TIF Summary Information

> [1] Example getting TIF summary information from DBFS (doesn't have to be a single file).

In [0]:
(spark.read
    .format("gdal")
    .option("raster_storage", "in-memory")
    .load("dbfs:/home/mjohns@databricks.com/stac/eo-series/data/alaska/B02/B02_S2B_MSIL2A_20210601T213529_R086_T06WXB_20210602T041754_20210601-213529.tif")
    .select(mos.rst_summary("tile"))).display()

> [2a] Example from 'tile' column in existing table / dataframe (`tile_df`).

In [0]:
summary_df = (
  tile_df
    .withColumn("summary_tile", mos.rst_summary("tile"))
    .selectExpr("summary_tile", "* except(summary_tile)")
)
summary_df.limit(5).display()

> [2b] Example parsing out band information from summary, in this example we count 'band.noDataValue' (which we previously standardized). __Note: there are other helper functions as well for metadata not shown, e.g. [rst_bandmetadata](https://databrickslabs.github.io/mosaic/api/raster-functions.html#rst_bandmetadata).__ We haven't yet stacked bands (will do in notebook 04), so dealing with single band TIFFs up to now.

In [0]:
# we use an exemplar to further parse structured metadata from the summary, in this case for bands. 
# - then we group by the numeric value for `band.noDataValue` and see if all is consistent
# - it is because we previously standardized it
example_band_obj = """[{"band":1,"block":[512,512],"type":"UInt16","colorInterpretation":"Gray","noDataValue":0.0,"overviews":[{"size":[5490,5490]},{"size":[2745,2745]},{"size":[1373,1373]},{"size":[687,687]},{"size":[344,344]}],"metadata":{}}]"""

(
    summary_df   
        .selectExpr("summary_tile:['bands'] as band_obj")
        .select(F.from_json("band_obj", F.schema_of_json(example_band_obj)).alias("bands"))
        .select(F.explode("bands").alias("band"))    
        .groupBy("band.noDataValue")
            .count()
).display()

## Tessellate @ H3 resolution=7

> To avoid re-computation per run, persisting the initial h3 tessellation as delta table. __Note: H3 Resolution 7 has an average area of 5km^2.__ This produces -700K tessellated tiles for Alaska and takes ~10 minutes (using notebook cluster config) if you want to run. We also OPTIMIZE by the H3 cell id ('index_id').

In [0]:
# Setting as variable for consistency across bands.
h3_resolution = 7

__Function to perform H3 tessellation for any/all bands.__

In [0]:
def gen_tessellate_tiled_band(
  tile_df, h3_res, tbl_name, h3_col="index_id", repartition_factor=3,
  do_try_open=False, do_display=True, do_overwrite=False, do_append=False
):
  """
  Generate H3 tessellation table from a tiled band dataframe.
  - Expects 'tile' column to be present.
  - If you have bad tiffs, set do_try_open=True
  Returns the resulting dataframe (from the spark table generated).
  """
  spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
  spark.conf.set("spark.sql.shuffle.partitions", 512)

  if do_overwrite:
    sql(f"""drop table if exists {tbl_name}""")
  if not spark.catalog.tableExists(tbl_name) or do_append:
    _tile_df = (
      tile_df
        .repartition(round(tile_df.count() / repartition_factor))
    )
    if do_try_open:
      _tile_df = _tile_df.filter(mos.rst_tryopen("tile"))
    (
     _tile_df
        .withColumn("tile", mos.rst_tessellate("tile", F.lit(h3_res)))
        .selectExpr(f"tile.index_id as {h3_col}", "*")
        .write
          .mode("append")
          .saveAsTable(tbl_name)
    )
  sql(f"""OPTIMIZE {tbl_name} ZORDER BY ({h3_col})""")
  h3_df = spark.table(tbl_name)
  if do_display:
    print(f"\n-- {tbl_name}")
    print(f"count? {h3_df.count():,}")
    h3_df.limit(10).display()
  return h3_df

In [0]:
h3_df = gen_tessellate_tiled_band(tile_df, h3_resolution, "band_b02_h3")

_We see there are ~430K individual H3 cells given that some tiles have cellids in common._

In [0]:
print(f"""count? { h3_df.groupBy("index_id").count().count():,}""") 
display (
  h3_df
  .groupBy("index_id")
    .count()
  .orderBy(F.desc("count"))
  .limit(25)
)

In [0]:
plot_h3_df = (
  h3_df
  .filter(col("item_id").contains("_R014_"))
  .groupBy("index_id")
    .count()
)
print(f"count? {plot_h3_df.count():,}")   

_Let's look at the ~50K cellids (with overlap counts) for 'item_ids' that contain `_R014_` (same as the bounding box earlier). Sidenote: since Alaska has some land on each side of the antimeridian, pull your mouse east to see the bulk of the results when rendering close to the boundary (not relevant for this output)._

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# plot_h3_df index_id h3 100_000

### Repeat for the other bands of interest

> Note: we are not doing an overwrite here, so if the table exists, it will just be returned.

__Band 03 (Green) tile and h3 tables.__

In [0]:
b03_df = (
  spark.read.table("band_b03")
  .filter(col('date') == F.lit(filter_date))
)
print(f"count? {b03_df.count():,}")
b03_df.orderBy("out_file_sz").limit(1).show(vertical=True) # <- show + limiting for ipynb only

In [0]:
gen_tiled_band_tbl(b03_df, "band_b03_tile")

In [0]:
gen_tessellate_tiled_band(spark.table("band_b03_tile"), h3_resolution, "band_b03_h3")

__Band 04 (Red) tile and h3 tables.__

In [0]:
b04_df = (
  spark.read.table("band_b04")
  .filter(col('date') == F.lit(filter_date))
)
print(f"count? {b04_df.count():,}")
b04_df.limit(10).limit(1).show(vertical=True) # <- show + limiting for ipynb only

In [0]:
gen_tiled_band_tbl(b04_df, "band_b04_tile")

In [0]:
gen_tessellate_tiled_band(spark.table("band_b04_tile"), h3_resolution, "band_b04_h3")

__Band 08 (NIR) tile and h3 tables.__

In [0]:
b08_df = (
  spark.read.table("band_b08")
  .filter(col('date') == F.lit(filter_date))
)
print(f"count? {b08_df.count():,}")
b08_df.limit(1).show(vertical=True) # <- show + limiting for ipynb only

In [0]:
gen_tiled_band_tbl(b08_df, "band_b08_tile")

In [0]:
gen_tessellate_tiled_band(spark.table("band_b08_tile"), h3_resolution, "band_b08_h3")

## K-Rings

> Once your rasters have been gridded, you can take advantage of calls such as K-Rings.

_We are just doing a kring around a H3 resolution 5 area in this example. Since our data is @ resolution 7, we can use the `h3_toparent` function available in product._

In [0]:
kring_cellid = h3_df.groupBy("index_id").count().orderBy(F.desc("count")).selectExpr("h3_toparent(index_id, 5)").first()[0]
kring_cellid

_Also, we are choosing to go out 3 rings from our center cell of interest in this example, using Mosaic's [grid_cellkringexplode](https://databrickslabs.github.io/mosaic/api/spatial-indexing.html#grid-cellkringexplode)._ __Note: We could also have used built-in product [h3_kring](https://docs.databricks.com/en/sql/language-manual/functions/h3_kring.html) for just the h3 cells around the provide one(s).__

In [0]:
kring_df = (
  h3_df
    .filter(dbf.h3_toparent("index_id", 5) == kring_cellid)
    .withColumn("kring", mos.grid_cellkringexplode("index_id", F.lit(3)))
    .groupBy("kring", "date")
      .agg(mos.rst_merge_agg("tile").alias("tile"))
)
print(f"count? {kring_df.count():,}")
kring_df.limit(10).display()

_Here is a plot of k-rings around our selected center cellid._

> Hint: zoom out to see all.

In [0]:
to_plot = (
  kring_df
  .select("kring")
)

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# to_plot kring h3

_Here is one of the rasters included in the kring data._

In [0]:
library.plot_raster(kring_df.select("tile.raster").first()[0])

## Raster to Timeseries projection

> __With the power of expression that rasterio provides + the power of distribution that mosaic provides__, we can easily convert rasters to numerical values with arbitrary mathematical complexity. Since all of our imagery is timestamped, our raster to number projection in effect is creating time series bound to H3 cells.

In [0]:
def rio_mean_band(dataset):
  try:
    return dataset.statistics(bidx = 1).mean
  except:
    return 0.0

measure_df = kring_df.withColumn(
  "rio_mean", library.rasterio_lambda("tile.raster", lambda dataset: rio_mean_band(dataset) )
)

In [0]:
daily_measure_df = (
  measure_df
    .selectExpr(
      "kring as index_id",
      "date",
      "rio_mean",
      "tile.raster"
    )
    .where("rio_mean > 0")
    .groupBy("index_id", "date")
    .agg(
      F.avg("rio_mean").alias("measure"),
      F.first("raster").alias("raster")
    )
)
daily_measure_df.limit(1).show(vertical=True) # <- show + limiting for ipynb only

_At this point our data is effectively became timeseries data and can be modeled as virtual IOT  devices that are fixed in spatial context. We can easily visualise data for individual dates in a spatial contex by leveraging our H3 locations._

In [0]:
df_06_01 = daily_measure_df.where("date == '2021-06-01'")

In [0]:
displayHTML("""<img src="..." width="25%">""")

_Uncomment the following within databricks for actual results._

In [0]:
# %%mosaic_kepler
# df_06_01 index_id h3 5000